# Predicting T-shirt size using the ANSUR II dataset
We will here try to predict a persons t-shirt size given the weight and height of the person. We will use the ANSUR II dataset which contains a lot of information about the physical attributes of a large number of people.
 
We will first try to map the persons in the dataset to a t-shirt size. It is hard to find a concise size chart for t-shirt so we will create our own, initial chart, based on these assumptions:
 
We will only look at two measurements, Shoulder Width and Chest Circumference.
 
Our first problem is that Shoulder Width is not one of the measurements taken in the dataset. But we have Biacromial Breadth which is the distance between the two acromion processes. We will assume that this is the same as Shoulder Width.
 
We will then have these initial rules:
 
| Size | Percentile |
|------|------------|
| XS   | 0-5        |
| S    | 5-25       |
| M    | 25-50      |
| L    | 50-75      |
| XL   | 75-90      |
| XXL  | 90-97      |
| XXXL | 97-100     |

### Inspect the data

In [1]:
import pandas as pd

In [2]:
female = pd.read_csv('./data/female.csv')
male = pd.read_csv('./data/male.csv')

In [3]:
print(f'For women we have (rows, columns) {female.shape}')
print(f'For men we have (rows, columns) {male.shape}')

For women we have (rows, columns) (1986, 108)
For men we have (rows, columns) (4082, 108)


### Checking the percentiles - Let us decide percentile of data

In [ ]:
def compute_percentile_ranges(column):
    ranges = [(0,5),(5,25),(25,50),(50,75),(75,90),(90,97),(97,100)]

    percentiles = { (low,high): (column.quantile(low/100), column.quantile(high/100)) for low, high in ranges}
    
    print(percentiles)

    counts = {}

    for r,(low, high) in percentiles.items():
        counts[r] = ((column >= low)& (column < high)).sum()
    return counts
print(compute_percentile_ranges(female['chestcircumference']))
print(compute_percentile_ranges(female['biacromialbreadth']))
print(compute_percentile_ranges(male['chestcircumference']))
print(compute_percentile_ranges(male['biacromialbreadth']))

{(0, 5): (695.0, 824.25), (5, 25): (824.25, 889.0), (25, 50): (889.0, 940.0), (50, 75): (940.0, 999.0), (75, 90): (999.0, 1057.0), (90, 97): (1057.0, 1117.45), (97, 100): (1117.45, 1266.0)}
None
{(0, 5): (283.0, 335.0), (5, 25): (335.0, 353.0), (25, 50): (353.0, 365.0), (50, 75): (365.0, 378.0), (75, 90): (378.0, 389.0), (90, 97): (389.0, 400.0), (97, 100): (400.0, 422.0)}
None
{(0, 5): (774.0, 922.0), (5, 25): (922.0, 996.0), (25, 50): (996.0, 1056.0), (50, 75): (1056.0, 1117.0), (75, 90): (1117.0, 1172.0), (90, 97): (1172.0, 1233.0), (97, 100): (1233.0, 1469.0)}
None
{(0, 5): (337.0, 384.0), (5, 25): (384.0, 403.0), (25, 50): (403.0, 415.0), (50, 75): (415.0, 428.0), (75, 90): (428.0, 441.0), (90, 97): (441.0, 452.5699999999997), (97, 100): (452.5699999999997, 489.0)}
None


### Generate the tshirt size chart

In [13]:
def compute_size_percentile_measurements(data, chest_column, shoulder_column):
    sizes = ['XS', 'S', 'M', 'L', 'XL', 'XXL', 'XXXL']
    ranges = [0,5,25,50,75,90,97]

    # Compute the values for each percentile for chest and shoulder
    chest_percentiles = {p:data[chest_column].quantile(p/100) for p in ranges}
    shoulder_percentiles = {p:data[shoulder_column].quantile(p/100) for p in ranges}
    #print(chest_percentiles)
    #print(shoulder_percentiles)

    # Map the tshrit sizes to the corresponding chest and shoulder measurements
    size_mappings = {}
    for i, size in enumerate(sizes):
        size_mappings[size] = {
            'Chest': int(chest_percentiles[ranges[i]]),
            'Shoulder': int(shoulder_percentiles[ranges[i]])
        }
    return size_mappings

print(compute_size_percentile_measurements(female,'chestcircumference','biacromialbreadth'))
print(compute_size_percentile_measurements(male,'chestcircumference','biacromialbreadth'))

{'XS': {'Chest': 695, 'Shoulder': 283}, 'S': {'Chest': 824, 'Shoulder': 335}, 'M': {'Chest': 889, 'Shoulder': 353}, 'L': {'Chest': 940, 'Shoulder': 365}, 'XL': {'Chest': 999, 'Shoulder': 378}, 'XXL': {'Chest': 1057, 'Shoulder': 389}, 'XXXL': {'Chest': 1117, 'Shoulder': 400}}
{'XS': {'Chest': 774, 'Shoulder': 337}, 'S': {'Chest': 922, 'Shoulder': 384}, 'M': {'Chest': 996, 'Shoulder': 403}, 'L': {'Chest': 1056, 'Shoulder': 415}, 'XL': {'Chest': 1117, 'Shoulder': 428}, 'XXL': {'Chest': 1172, 'Shoulder': 441}, 'XXXL': {'Chest': 1233, 'Shoulder': 452}}


In [ ]:
female_size = {
    'XS': {'Chest': 695, 'Shoulder': 283}, 
    'S': {'Chest': 824, 'Shoulder': 335}, 
    'M': {'Chest': 889, 'Shoulder': 353}, 
    'L': {'Chest': 940, 'Shoulder': 365}, 
    'XL': {'Chest': 999, 'Shoulder': 378}, 
    'XXL': {'Chest': 1057, 'Shoulder': 389}, 
    'XXXL': {'Chest': 1117, 'Shoulder': 400}
    }

male_sizes = {
    'XS': {'Chest': 774, 'Shoulder': 337}, 
    'S': {'Chest': 922, 'Shoulder': 384}, 
    'M': {'Chest': 996, 'Shoulder': 403}, 
    'L': {'Chest': 1056, 'Shoulder': 415}, 
    'XL': {'Chest': 1117, 'Shoulder': 428}, 
    'XXL': {'Chest': 1172, 'Shoulder': 441}, 
    'XXXL': {'Chest': 1233, 'Shoulder': 452}
    }